In [119]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import nltk 
from nltk.corpus import gutenberg
nltk.download('gutenberg')
nltk.download('punkt')
import os

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/infres/pmauduit-21/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/infres/pmauduit-21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [120]:
device = torch.device('cuda:2'if torch.cuda.is_available() else "cpu") # Use GPU if available
print('device:', device)
print('current directory:', os.getcwd())

device: cuda:2
current directory: /home/infres/pmauduit-21/repository/DD2424-DL-Text_Gen_Project


### Import Data

In [121]:
alice = gutenberg.raw('carroll-alice.txt')
header = alice.find("CHAPTER I.")
alice = alice[header:]
# raw_text = alice.lower()
raw_text = alice
chars = sorted(list(set(raw_text)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
n_vocab = len(chars)
train_text = raw_text[:-22067]
val_text = raw_text[-22067:-11681]
test_text = raw_text[-11681:]

print('vocab size:', n_vocab)

vocab size: 69


### Generating text

In [122]:
class CharModel(nn.Module):
    def __init__(self, n_vocab, embedding_dim=128, hidden_size=256, num_layers=2, dropout=0, norm=False, type = 'lstm'):
        super().__init__()
        if embedding_dim > 1:
            self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=embedding_dim)   
        if type == 'lstm':
            self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        elif type == 'gru':
            self.lstm = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        elif type == 'rnn':
            self.lstm = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        if norm:
            self.norm = nn.LayerNorm(hidden_size)
        self.linear = nn.Linear(hidden_size, n_vocab)

    def forward(self, x):
        # Convert input to Long type
        if hasattr(self, 'embedding'):
            x = x.long().squeeze(-1)
            x = self.embedding(x)
        else:
            x = x/float(n_vocab)
            if x.dim() == 2:
                x.unsqueeze_(-1) # Problem ???
        x, _ = self.lstm(x)
        # Take only the last output
        if x.dim() == 2:
            x = x.unsqueeze(0) # Problem with batch size = 1 during inference
        x = x[:, -1, :]
        # Normalize
        if hasattr(self, 'norm'):
            x = self.norm(x)
        x = self.linear(self.dropout(x))
        return x
    

In [123]:
# model = CharModel(n_vocab, embedding_dim=100, hidden_size=512, num_layers=2, dropout=0.3, norm=True, type='lstm').to(device)
model = CharModel(n_vocab, embedding_dim=100, hidden_size=256, num_layers=2, dropout=0.3, norm=True, type='lstm').to(device)
seq_length = 100

# best_model = torch.load("lstm_norm_True_raw_huge.pt")
best_model = torch.load("lstm_norm_True_raw.pt")

n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

model.eval()

CharModel(
  (embedding): Embedding(69, 100)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (linear): Linear(in_features=256, out_features=69, bias=True)
)

In [124]:
def apply_temperature_scaling(logits, temperature):
    logits = logits / temperature
    probs = torch.softmax(logits, dim=-1)
    return probs

def nucleus_sampling(probs, p, device):
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)    
    cutoff_index = torch.searchsorted(cumulative_probs, torch.tensor(p, device=device)).item() + 1
    top_probs = sorted_probs[:cutoff_index]
    top_indices = sorted_indices[:cutoff_index]    
    top_probs = top_probs / torch.sum(top_probs)    
    sampled_index = torch.multinomial(top_probs, 1).item()    
    return top_indices[sampled_index].item()

def generate_from_text(temperature=None, p = None, prompt=None, doPrint = False):
    if prompt is None:
        start = np.random.randint(0, len(raw_text)-seq_length)
        text_prompt = raw_text[start:start+seq_length]
        pattern = [char_to_int[c] for c in text_prompt]
        if doPrint:
            print('Prompt: "%s"' % text_prompt)
            print('-'*100)
    else:
        pattern = [char_to_int[c] for c in prompt]
        if doPrint:
            print('Prompt: "%s"' % prompt)
            print('-'*100)
    model.eval()
    generated_text = ""
    if prompt is not None:
        if doPrint:
            print(prompt, end="")
        generated_text += prompt
    with torch.no_grad():
        for i in range(1000):
            x = np.reshape(pattern, (1, len(pattern)))
            # x = torch.tensor(x, dtype=torch.float32)
            x = torch.tensor(x, dtype=torch.long)
            logits = model(x.to(device))
            if temperature is not None:
                scaled_probs = apply_temperature_scaling(logits, temperature)
                if p is not None:                
                    scaled_probs = scaled_probs.squeeze()
                    index = nucleus_sampling(scaled_probs, p, device)
                else:
                    index = torch.multinomial(scaled_probs, 1).item()
            else:
                index = int(logits.argmax())
            result = int_to_char[index]
            generated_text += result
            if doPrint:
                print(result, end="")
            pattern.append(index)
            if len(pattern) > seq_length:
                pattern = pattern[1:]
    if doPrint:
        print()
    print("Done.")    
    return generated_text

### Evaluation

In [125]:
reference_text = alice

#### TTR score (Type Token Ratio: measure of vocabulary variation)

The higher the TTR score, the more diverse the vocabulary used in the text.

Temperature sampling gives higher TTR scores than nucleus sampling, but text is less coherent.

In [126]:
def calculate_ttr(text):
    words = text.split()
    unique_words = set(words)
    ttr = len(unique_words) / len(words)
    return ttr

In [127]:
greedy_text = generate_from_text()
temperature_text = generate_from_text(temperature=0.8)
nucleus_text = generate_from_text(temperature=0.8, p=0.9)

print("Greedy Text TTR:", calculate_ttr(greedy_text))
print("Temperature Text TTR:", calculate_ttr(temperature_text))
print("Nucleus Text TTR:", calculate_ttr(nucleus_text))


Done.
Done.
Done.
Greedy Text TTR: 0.2361111111111111
Temperature Text TTR: 0.7
Nucleus Text TTR: 0.6089108910891089


#### Perplexity (how well the model predicts a word sequence)

The lower the perplexity, the better the model predicts the next word.

Greedy sampling gives the lowest perplexity, but text is also the least diverse.

In [128]:
def calculate_perplexity(text, model, tokenizer, device):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    lls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone().to(device)
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs.loss * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    return ppl.item()

In [129]:
# Reference text for BLEU score calculation
reference_text = raw_text[-1000:]

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

perplexity_greedy = calculate_perplexity(greedy_text, gpt2_model, tokenizer, device)
print(f"Perplexity greedy: {perplexity_greedy}")

perplexity_temperature = calculate_perplexity(temperature_text, gpt2_model, tokenizer, device)
print(f"Perplexity temperature: {perplexity_temperature}")

perplexity_nucleus = calculate_perplexity(nucleus_text, gpt2_model, tokenizer, device)
print(f"Perplexity nucleus: {perplexity_nucleus}")

perplexity = calculate_perplexity(reference_text, gpt2_model, tokenizer, device)
print(f"Perplexity reference: {perplexity}")

Perplexity greedy: 3.982386589050293
Perplexity temperature: 112.10043334960938
Perplexity nucleus: 98.43819427490234
Perplexity reference: 50.75355911254883


#### Spell Check

We clean the text by removing special characters. 

In [130]:
import re
from spellchecker import SpellChecker

def calculate_correctly_spelled_percentage(text):
    # Initialize the spell checker
    spell = SpellChecker()

    # Remove punctuation and split the text into words
    words = re.findall(r'\b\w+\b', text)

    # Check each word for correctness
    misspelled = spell.unknown(words)

    # Calculate the number of correctly spelled words
    correctly_spelled_count = len(words) - len(misspelled)

    # Calculate the percentage of correctly spelled words
    if len(words) == 0:
        return 0.0  # Avoid division by zero if there are no words
    correctly_spelled_percentage = (correctly_spelled_count / len(words)) * 100

    return correctly_spelled_percentage

greedy_text_percentage = calculate_correctly_spelled_percentage(greedy_text)
print(f"Greedy Text Percentage of correctly spelled words: {greedy_text_percentage:.2f}%")

temperature_text_percentage = calculate_correctly_spelled_percentage(temperature_text)
print(f"Temperature Text Percentage of correctly spelled words: {temperature_text_percentage:.2f}%")

nucleus_text_percentage = calculate_correctly_spelled_percentage(nucleus_text)
print(f"Nucleus Text Percentage of correctly spelled words: {nucleus_text_percentage:.2f}%")

reference_text_percentage = calculate_correctly_spelled_percentage(reference_text)
print(f"Reference Text Percentage of correctly spelled words: {reference_text_percentage:.2f}%")


Greedy Text Percentage of correctly spelled words: 99.55%
Temperature Text Percentage of correctly spelled words: 94.50%
Nucleus Text Percentage of correctly spelled words: 97.66%
Reference Text Percentage of correctly spelled words: 99.46%


#### BLEU score

In [169]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu_score(reference_text, generated_text):
    # Tokenize the reference and generated text
    reference_tokens = nltk.word_tokenize(reference_text)
    generated_tokens = nltk.word_tokenize(generated_text)

    # Reference text should be a list of lists of tokens
    reference_list = [reference_tokens]

    # Use SmoothingFunction to handle very short texts
    smoothing_function = SmoothingFunction().method4

    # Compute BLEU score
    bleu_score = sentence_bleu(reference_list, generated_tokens, smoothing_function=smoothing_function)

    return bleu_score

# Example usage
# reference_text = alice[1000:2000]
reference_text = alice[-1000:]

generated_text = nucleus_text
# generated_text = temperature_text
# generated_text = greedy_text

# generated_text = alice[12000:13000]

bleu_score = calculate_bleu_score(reference_text, generated_text) * 100
print(f"Average BLEU Score: {bleu_score:.2f}")


Average BLEU Score: 1.19
